# Aprendizaje Automático - Intermedio

## Valores ausentes

### Introducción

En este tutorial, aprenderemos tres enfoques para tratar los valores ausentes. Luego compararemos la efectividad de estos enfoques en un conjunto de datos del mundo real.

Hay muchas formas en las que los datos pueden terminar con valores ausentes. Por ejemplo,

+ Una casa de 2 dormitorios no incluirá un valor para el tamaño de un tercer dormitorio.
+ Un encuestado puede optar por no compartir sus ingresos.

La mayoría de las librerías de aprendizaje automático (incluido scikit-learn) dan un error si intentamos construir un modelo utilizando datos con valores faltantes. Por lo tanto, deberemos elegir una de las estrategias a continuación.

### Tres enfoques

#### 1) Opción simple: Eliminar las columnas con valores faltantes

La opción más simple es eliminar las columnas con valores faltantes.

![Opción1](./images/valores_faltantes_1.png)

A menos que falten la mayoría de los valores en las columnas descartadas, el modelo pierde acceso a mucha información (¡potencialmente útil!) con este enfoque. Como ejemplo extremo, considere un conjunto de datos con 10.000 filas, donde a una columna importante le falta una sola entrada. ¡Este enfoque eliminaría la columna por completo!

#### 2) Una mejor opción: Imputación

La **imputación** completa los valores faltantes con algún número. Por ejemplo, podemos completar el valor medio a lo largo de cada columna.

![Valores_faltantes_2](./images/valores_faltantes_2.png)

El valor imputado no será exactamente correcto en la mayoría de los casos, pero generalmente conduce a modelos más precisos de los que obtendría al eliminar la columna por completo.

#### 3) Una extesión a la imputacion

La imputación es el enfoque estándar y generalmente funciona bien. Sin embargo, los valores imputados pueden estar sistemáticamente por encima o por debajo de sus valores reales (que no se recopilaron en el conjunto de datos). O las filas con valores ausentes pueden ser únicas de alguna otra manera. En ese caso, tu modelo haría mejores predicciones al considerar qué valores faltaban originalmente.

![valores_faltantes_3](./images/valores_faltantes_3.png)

En este enfoque imputamos los valores ausentes como antes. Y, además, para cada columna con entradas faltantes en el conjunto de datos original, agregamos una nueva columna que muestra la ubicación de las entradas imputadas.

En algunos casos, esto mejorará significativamente los resultados. En otros casos, no ayuda en absoluto.

### Ejemplo

En el ejemplo, trabajaremos con el conjunto de datos de [Melbourne Housing](https://www.kaggle.com/dansbecker/melbourne-housing-snapshot). Nuestro modelo utilizará información como la cantidad de habitaciones y el tamaño del terreno para predecir el precio de la vivienda.

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Carga los data
data = pd.read_csv('./input/melbourne-housing-snapshot/melb_data.csv')

# Selecciona el objetivo
y = data.Price

# Para mantener las cosas simples, usaremos solo predictores numéricos
melb_predictors = data.drop(['Price'], axis=1)
X = melb_predictors.select_dtypes(exclude=['object'])

# divide los datos en entrenamiento y validación
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=0)

#### Define la función para medir la calidad de cada enfoque

Definimos una función `score_dataset()` para comparar los diferentes enfoques de tratar los valores perdidos. Esta función informa el error absoluto medio (MAE) de un modelo de random forest.

In [2]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

# Función para comparar diferentes enfoques
def score_dataset(X_train, X_valid, y_train, y_valid):
    model = RandomForestRegressor(n_estimators=10, random_state=0)
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    return mean_absolute_error(y_valid, preds)

#### Puntuación del Enfoque 1 (Eliminar columnas con valores ausentes)

Como estamos trabajando con conjuntos de entrenamiento y validación, tenemos cuidado de eliminar las mismas columnas en ambos dataframes.

In [3]:
# Obtiene los nombres de las columnas con valores ausentes
cols_with_missing = [col for col in X_train.columns
                     if X_train[col].isnull().any()]

# Elimina las columnas en datos de entrenamiento y validación
reduced_X_train = X_train.drop(cols_with_missing, axis=1)
reduced_X_valid = X_valid.drop(cols_with_missing, axis=1)

print("MAE de Enfoque 1 (Eliminar columnas con valores ausentes):")
print(score_dataset(reduced_X_train, reduced_X_valid, y_train, y_valid))

MAE de Enfoque 1 (Eliminar columnas con valores ausentes):
183550.22137772635


#### Puntuación del Enfoque 2 (Imputación)

Usamos `SimpleImputer` para reemplazar los valores faltantes con el valor medio a lo largo de cada columna.

Aunque es simple, completar el valor medio generalmente funciona bastante bien (pero esto varía según el conjunto de datos). Si bien los estadísticos han experimentado formas más complejas de determinar los valores imputados (como la **imputación de regresión**, por ejemplo), las estrategias complejas generalmente no brindan ningún beneficio adicional.

In [4]:
from sklearn.impute import SimpleImputer

# Imputación
my_imputer = SimpleImputer()
imputed_X_train = pd.DataFrame(my_imputer.fit_transform(X_train))
imputed_X_valid = pd.DataFrame(my_imputer.transform(X_valid))

# La imputación elimina los nombres de las columnas; los recuperamos
imputed_X_train.columns = X_train.columns
imputed_X_valid.columns = X_valid.columns

print("MAE de Enfoque 2 (Imputación):")
print(score_dataset(imputed_X_train, imputed_X_valid, y_train, y_valid))

MAE de Enfoque 2 (Imputación):
178166.46269899711


Vemos que el Enfoque 2 tiene un MAE más bajo que el Enfoque 1, por lo que el Enfoque 2 se ejecutó mejor en este conjunto de datos.